In [1]:
! pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.9 MB/s eta 0:00:00a 0:00:01


In [2]:
! pip install mediapipe

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.5.0-py3-none-any.whl.metadata (22 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached protobuf-4.25.6-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 7.0 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
Using cached protobuf-4.25.6-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_univ

In [2]:
import cv2
import mediapipe as mp
import time 

In [3]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

I0000 00:00:1743087417.457531 16510701 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743087417.562839 16557939 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743087417.584458 16557941 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
cap = cv2.VideoCapture("PoseVids/Squat2.MOV")


# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame_width, frame_height))

pTime = 0
start = time.time()
while True:
    success, img = cap.read()
    if not success:
        break
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # mediapipe uses RGB not GBR 
    results = pose.process(imgRGB)
    cTime = time.time()
    # print(results.pose_landmarks)
    if results.pose_landmarks:
        mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = img.shape
            # print(id, lm) 
            cx, cy = int(lm.x*w), int(lm.y*h)# landmarks are in ratio of image size, multiply by width and height to get pixel values
            cv2.circle(img, (cx,cy), 10, (255,0,0), cv2.FILLED)
    fps = 1/(cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(cTime-start), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
    cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

W0000 00:00:1743087419.547098 16557941 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
